In [10]:
import cudaq
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Function
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import shap
from lime.lime_tabular import LimeTabularExplainer

cudaq.set_target("qpp-cpu")  # or "nvidia" if you have CUDA-Q GPU build
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [14]:
# ---------------------
# Quantum Function (2 qubits)
# ---------------------
class QuantumFunction(torch.autograd.Function):
    @staticmethod
    def forward(ctx, thetas: torch.Tensor, quantum_circuit, shift: float):
        ctx.shift = shift
        ctx.quantum_circuit = quantum_circuit

        # Run kernel → expectation values
        exp_vals = ctx.quantum_circuit.run(thetas).reshape(-1, 1)
        ctx.save_for_backward(thetas, exp_vals)
        return exp_vals

    @staticmethod
    def backward(ctx, grad_output):
        thetas, _ = ctx.saved_tensors
        grads = torch.zeros_like(thetas)
        s = ctx.shift
        qc = ctx.quantum_circuit

        for j in range(thetas.shape[1]):
            tp = thetas.clone(); tp[:, j] += s
            tm = thetas.clone(); tm[:, j] -= s
            exp_p = qc.run(tp).view(-1,1)
            exp_m = qc.run(tm).view(-1,1)
            dE = (exp_p - exp_m) / (2.0 * s)
            grads[:, j:j+1] = dE * grad_output
        return grads, None, None

# ---------------------
# Quantum Circuit Wrapper
# ---------------------
class QuantumCircuitWrapper:
    def __init__(self, qubit_count: int, hamiltonian: cudaq.SpinOperator):
        @cudaq.kernel
        def kernel(qubit_count: int, thetas: np.ndarray):
            q = cudaq.qvector(qubit_count)

            # Encode qubit 0
            ry(thetas[0], q[0])
            rx(thetas[1], q[0])

            # Encode qubit 1
            ry(thetas[2], q[1])
            rx(thetas[3], q[1])

            # Entangle
            cx(q[0], q[1])

        self.kernel = kernel
        self.qubit_count = qubit_count
        self.hamiltonian = hamiltonian

    def run(self, theta_vals: torch.Tensor) -> torch.Tensor:
        theta_np = theta_vals.detach().cpu().numpy()
        exps = []
        for i in range(theta_np.shape[0]):
            res = cudaq.observe(
                self.kernel,
                self.hamiltonian,
                self.qubit_count,
                np.array(theta_np[i], dtype=np.float64)
            )
            exps.append(res.expectation())
        return torch.tensor(exps, dtype=torch.float32, device=device)

# ---------------------
# Quantum Layer (2 qubits)
# ---------------------
class QuantumLayer(nn.Module):
    def __init__(self, qubit_count: int, hamiltonian, shift: float):
        super().__init__()
        self.quantum_circuit = QuantumCircuitWrapper(qubit_count, hamiltonian)
        self.shift = shift

    def forward(self, x: torch.Tensor):
        # Apply custom autograd function
        return QuantumFunction.apply(x, self.quantum_circuit, self.shift)

# ---------------------
# Residual QNN with 2-qubit quantum block
# ---------------------
class ResidualQNN(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 64)
        self.skip_linear = nn.Linear(128, 64)

        # Quantum block (2 qubits → 4 params per sample)
        self.fc_map = nn.Linear(64, 4)  # map classical → 4 angles
        self.quantum = QuantumLayer(
            qubit_count=2,
            hamiltonian=(cudaq.spin.z(0) + cudaq.spin.z(1)) / 2.0,
            shift=np.pi / 2
        )

        self.fc_out = nn.Linear(1, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        residual = self.skip_linear(x)
        x = torch.relu(self.fc3(x))
        x = x + residual

        # compress 64 → 4 parameters (for 2 qubits)
        angles = self.fc_map(x)
        q_out = self.quantum(angles).view(-1, 1)

        return self.fc_out(q_out)


In [15]:
# ---------------------
#   Data (predict Sint)
# ---------------------
def load_data(path):
    return pd.read_csv(path)

dataset_path = "../../data/data.csv"
data = load_data(dataset_path)

# numeric fill
num_cols = data.select_dtypes(include=[np.number]).columns
data[num_cols] = data[num_cols].fillna(data[num_cols].mean())

# Features: drop ID-like and target columns
drop_cols = ['ID', 'Unnamed: 0', 'Sint', 'e_Sint']
feature_cols = [c for c in data.columns if c not in drop_cols]
X = data[feature_cols].values.astype(np.float32)

# Target: Sint (1D)
y = data['Sint'].values.astype(np.float32).reshape(-1, 1)

# Normalize
scaler_X = StandardScaler()
scaler_y = StandardScaler()

X = scaler_X.fit_transform(X).astype(np.float32)
y = scaler_y.fit_transform(y).astype(np.float32)  # train on scaled y for stability

# Tensors
X = torch.tensor(X, dtype=torch.float32, device=device)
y = torch.tensor(y, dtype=torch.float32, device=device)

# Split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.1, random_state=42
)


In [17]:
# ---------------------
#   Training
# ---------------------
model = ResidualQNN(input_dim=X.shape[1]).to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)
loss_function = nn.MSELoss().to(device)

epochs = 20
losses = []
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    y_hat = model(X_train)
    loss = loss_function(y_hat, y_train)
    loss.backward()
    optimizer.step()

    losses.append(loss.item())
    print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item():.6f}, Std Dev: {np.std(losses):.6f}")


Epoch 1/20, Loss: 1.630602, Std Dev: 0.000000
Epoch 2/20, Loss: 1.628107, Std Dev: 0.001247
Epoch 3/20, Loss: 1.571103, Std Dev: 0.027479
Epoch 4/20, Loss: 1.506126, Std Dev: 0.050862
Epoch 5/20, Loss: 1.428799, Std Dev: 0.076959
Epoch 6/20, Loss: 1.326630, Std Dev: 0.109770
Epoch 7/20, Loss: 1.213537, Std Dev: 0.146537
Epoch 8/20, Loss: 1.120244, Std Dev: 0.179811
Epoch 9/20, Loss: 1.075212, Std Dev: 0.202588
Epoch 10/20, Loss: 1.085574, Std Dev: 0.212650
Epoch 11/20, Loss: 1.103379, Std Dev: 0.215620
Epoch 12/20, Loss: 1.098939, Std Dev: 0.216538
Epoch 13/20, Loss: 1.081472, Std Dev: 0.217203
Epoch 14/20, Loss: 1.064757, Std Dev: 0.217728
Epoch 15/20, Loss: 1.049580, Std Dev: 0.218124
Epoch 16/20, Loss: 1.039027, Std Dev: 0.218203
Epoch 17/20, Loss: 1.036216, Std Dev: 0.217661
Epoch 18/20, Loss: 1.035148, Std Dev: 0.216611
Epoch 19/20, Loss: 1.032148, Std Dev: 0.215297
Epoch 20/20, Loss: 1.029389, Std Dev: 0.213800


In [18]:
# ---------------------
#   Evaluation
# ---------------------
model.eval()
with torch.no_grad():
    y_hat_test = model(X_test)

# to numpy (scaled space)
y_test_np = y_test.detach().cpu().numpy()
y_hat_np  = y_hat_test.detach().cpu().numpy()

# Metrics (scaled)
r2_scaled  = r2_score(y_test_np, y_hat_np)
mae_scaled = mean_absolute_error(y_test_np, y_hat_np)
rmse_scaled = np.sqrt(mean_squared_error(y_test_np, y_hat_np))
mse_scaled  = mean_squared_error(y_test_np, y_hat_np)

print("\n--- Evaluation (scaled) ---")
print(f"R^2:  {r2_scaled:.4f}")
print(f"MAE:  {mae_scaled:.4f}")
print(f"RMSE: {rmse_scaled:.4f}")
print(f"MSE:  {mse_scaled:.4f}")

# Metrics in original units
y_test_orig = scaler_y.inverse_transform(y_test_np)
y_pred_orig = scaler_y.inverse_transform(y_hat_np)

r2_orig  = r2_score(y_test_orig, y_pred_orig)
mae_orig = mean_absolute_error(y_test_orig, y_pred_orig)
rmse_orig = np.sqrt(mean_squared_error(y_test_orig, y_pred_orig))
mse_orig  = mean_squared_error(y_test_orig, y_pred_orig)

range_y = y_test_orig.max() - y_test_orig.min()
mae_acc  = (1 - mae_orig / range_y) * 100 if range_y > 0 else np.nan
rmse_acc = (1 - rmse_orig / range_y) * 100 if range_y > 0 else np.nan
mse_acc  = (1 - mse_orig / range_y) * 100 if range_y > 0 else np.nan

print("\n--- Evaluation (original Sint units) ---")
print(f"R^2:  {r2_orig:.4f}")
print(f"MAE:  {mae_orig:.4f} | Accuracy: {mae_acc:.2f}%")
print(f"RMSE: {rmse_orig:.4f} | Accuracy: {rmse_acc:.2f}%")
print(f"MSE:  {mse_orig:.4f} | Accuracy: {mse_acc:.2f}%")


--- Evaluation (scaled) ---
R^2:  0.0637
MAE:  0.2040
RMSE: 0.5820
MSE:  0.3387

--- Evaluation (original Sint units) ---
R^2:  0.0637
MAE:  2.0747 | Accuracy: 96.51%
RMSE: 5.9184 | Accuracy: 90.06%
MSE:  35.0277 | Accuracy: 41.15%
